In [1]:
import numpy as np
import mplhep as hep
import matplotlib.pyplot as plt
import uproot, os, sys
import awkward as ak
# Get the notebook directory
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
# Add the project root to sys.path
sys.path.append(os.path.join(notebook_dir, ".."))
from utils.branches import get_branches
from utils.plot import plot_data
from utils.constants import trigcut, truthpkk
from matplotlib import rcParams
import matplotlib as mpl
plt.style.use(hep.style.LHCb1)
config = {"mathtext.fontset":'stix'}
rcParams.update(config)

In [2]:
plt.rcParams.update({
    # Keep the font family settings for LHCb style
    "font.family": "serif",
    "font.serif": ["Times", "Computer Modern Roman", "DejaVu Serif"],
    
    # # Increase only the size-related parameters
    # "figure.figsize": (15, 10),  # Larger figure
    # "figure.dpi": 100,          # Screen display
    # "savefig.dpi": 300,         # Saved figure resolution
    
    # # # Increase font sizes while keeping LHCb style
    "font.size": 12,            # Base font size (increase from default)
    "axes.titlesize": 12,       # Title size
    "axes.labelsize": 10,       # Axis label size
    "xtick.labelsize": 12,      # X tick label size
    "ytick.labelsize": 12,      # Y tick label size
    "legend.fontsize": 12       # Legend font size
})


## Loading MC data

In [3]:
from utils.data_loader import load_mc_data
decay = r"$B^+\to \bar{\Lambda}^0pK^+K^-$ MC"


In [4]:


mc_dd = load_mc_data(
    mc_path="/share/lazy/Mohamed/Bu2LambdaPPP/MC/DaVinciTuples/restripped.MC",
    decay_mode="L0barPKpKm",
    particles=["h1", "h2", "p"],
    additional_branches=[
        "p_MC15TuneV1_ProbNNp",
        "h1_MC15TuneV1_ProbNNk",
        "h2_MC15TuneV1_ProbNNk",
        "Bu_TRUEID"
    ],
    tracks=["DD"], # ["DD", "LL"],
    cuts=trigcut + truthpkk
)

mc_ll = load_mc_data(
    mc_path="/share/lazy/Mohamed/Bu2LambdaPPP/MC/DaVinciTuples/restripped.MC",
    decay_mode="L0barPKpKm",
    particles=["h1", "h2", "p"],
    additional_branches=[
        "p_MC15TuneV1_ProbNNp",
        "h1_MC15TuneV1_ProbNNk",
        "h2_MC15TuneV1_ProbNNk",
        "Bu_TRUEID"
    ],
    tracks=["LL"], # ["DD", "LL"],
    cuts=trigcut + truthpkk
)

MC Files being processed with trees ['DD']: ['/share/lazy/Mohamed/Bu2LambdaPPP/MC/DaVinciTuples/restripped.MC/MC16MDBu2L0barPKpKm.root:B2L0barPKpKm_DD/DecayTree', '/share/lazy/Mohamed/Bu2LambdaPPP/MC/DaVinciTuples/restripped.MC/MC16MUBu2L0barPKpKm.root:B2L0barPKpKm_DD/DecayTree', '/share/lazy/Mohamed/Bu2LambdaPPP/MC/DaVinciTuples/restripped.MC/MC17MDBu2L0barPKpKm.root:B2L0barPKpKm_DD/DecayTree', '/share/lazy/Mohamed/Bu2LambdaPPP/MC/DaVinciTuples/restripped.MC/MC17MUBu2L0barPKpKm.root:B2L0barPKpKm_DD/DecayTree', '/share/lazy/Mohamed/Bu2LambdaPPP/MC/DaVinciTuples/restripped.MC/MC18MDBu2L0barPKpKm.root:B2L0barPKpKm_DD/DecayTree', '/share/lazy/Mohamed/Bu2LambdaPPP/MC/DaVinciTuples/restripped.MC/MC18MUBu2L0barPKpKm.root:B2L0barPKpKm_DD/DecayTree']
MC Branches being read: ['h1_P', 'h1_PT', 'h1_PE', 'h1_PX', 'h1_PY', 'h1_PZ', 'h1_ID', 'h1_TRACK_Type', 'h1_IPCHI2_OWNPV', 'h2_P', 'h2_PT', 'h2_PE', 'h2_PX', 'h2_PY', 'h2_PZ', 'h2_ID', 'h2_TRACK_Type', 'h2_IPCHI2_OWNPV', 'p_P', 'p_PT', 'p_PE', 'p_

In [5]:
# from utils.mc_kpkm import *

# output_dir = "output_['DD']/"

# # Run all fitting models
# print("\n=== Fitting with Gaussian + Poly2 model ===")
# gp_results = extract_signal_shape_with_background(
#     mc_selection,
#     true_id_branch='Bu_TRUEID',
#     mass_branch='Bu_MM',
#     output_dir=output_dir
# )

# print("\n=== Fitting with Crystal Ball + Poly2 model ===")
# cb_results = extract_signal_shape_cb_with_background(
#     mc_selection,
#     true_id_branch='Bu_TRUEID',
#     mass_branch='Bu_MM',
#     output_dir=output_dir
# )

# print("\n=== Fitting with Double Gaussian + Poly2 model ===")
# dg_results = extract_signal_shape_double_gaussian_with_background(
#     mc_selection,
#     true_id_branch='Bu_TRUEID',
#     mass_branch='Bu_MM',
#     output_dir=output_dir
# )

# print("\n=== Fitting with Double Crystal Ball + Poly2 model ===")
# dcb_results = extract_signal_shape_double_cb_with_background(
#     mc_selection,
#     true_id_branch='Bu_TRUEID',
#     mass_branch='Bu_MM',
#     output_dir=output_dir
# )

# # Compare all models
# print("\nModel comparison by χ²/ndf:")
# print(f"Gaussian + Poly2: {gp_results['chi2_ndf']:.3f}")
# print(f"Crystal Ball + Poly2: {cb_results['chi2_ndf']:.3f}")
# print(f"Double Gaussian + Poly2: {dg_results['chi2_ndf']:.3f}")
# print(f"Double Crystal Ball + Poly2: {dcb_results['chi2_ndf']:.3f}")

# # Create a dictionary of all results for further analysis
# all_results = {
#     "gaussian_poly2": gp_results,
#     "crystal_ball_poly2": cb_results,
#     "double_gaussian_poly2": dg_results,
#     "double_crystal_ball_poly2": dcb_results
# }

# # Find best model based on χ²/ndf
# best_model = min(all_results.items(), key=lambda x: x[1]['chi2_ndf'])
# print(f"\nBest model: {best_model[0]} with χ²/ndf = {best_model[1]['chi2_ndf']:.3f}")

# # Save all results to a JSON file for reference
# with open(f"{output_dir}/model_comparison.json", 'w') as f:
#     comparison = {
#         model_name: {"chi2_ndf": params["chi2_ndf"]} 
#         for model_name, params in all_results.items()
#     }
#     json.dump(comparison, f, indent=4)

In [6]:
def select(data, track_type='LL'):
    """
    Apply selection cuts to B+ → Λ0 h1 h2 samples incrementally
    
    Parameters:
    -----------
    data : awkward.Array
        Events data
    track_type : str
        Track type, either 'LL' (Long-Long) or 'DD' (Downstream-Downstream)
    
    Returns:
    --------
    awkward.Array
        Selected events after applying all cuts
    """
    # Keep track of initial count for reporting
    initial_count = len(data)
    
    # Apply cuts one at a time, printing progress if desired
    print(f"Initial events: {initial_count}")
    
    # ===== p (Proton) Cuts =====
    data = data[data['p_MC15TuneV1_ProbNNp'] > 0.05]
    print(f"After proton prob cut: {len(data)}/{initial_count} ({len(data)/initial_count:.2%})")
    
    # ===== Λ0 Cuts =====
    # Delta Z cut (difference between Lambda decay vertex and primary vertex)
    data = data[(data['L0_ENDVERTEX_Z'] - data['L0_OWNPV_Z']) > 20]
    print(f"After delta Z cut: {len(data)}/{initial_count} ({len(data)/initial_count:.2%})")
    
    # Lambda flight distance chi2
    try:
        data = data[data['L0_FDCHI2_OWNPV'] > 45]
    except:
        # Try alternative field name
        data = data[data['L0_FDCHI2_ORIVX'] > 45]
    print(f"After fd chi2 cut: {len(data)}/{initial_count} ({len(data)/initial_count:.2%})")
    
    # Lambda mass window
    data = data[(data['L0_M'] - 1115.6 < 6) & (data['L0_M'] - 1115.6 > -6)]
    print(f"After lambda mass cut: {len(data)}/{initial_count} ({len(data)/initial_count:.2%})")
    
    # Lambda proton ProbNN
    data = data[data['Lp_MC15TuneV1_ProbNNp'] > 0.2]
    print(f"After lambda proton prob cut: {len(data)}/{initial_count} ({len(data)/initial_count:.2%})")
    
    # ===== h1 and h2 (Kaon) Cuts =====
    # KK product
    try:
        data = data[(data['h1_ProbNNk'] * data['h2_ProbNNk']) > 0.04]
    except:
        # Try alternative field names
        data = data[(data['h1_MC15TuneV1_ProbNNk'] * data['h2_MC15TuneV1_ProbNNk']) > 0.04]
    print(f"After KK product cut: {len(data)}/{initial_count} ({len(data)/initial_count:.2%})")
    
    # ===== B⁺ Cuts =====
    # B PT cut
    data = data[data['Bu_PT'] > 3000]
    print(f"After B PT cut: {len(data)}/{initial_count} ({len(data)/initial_count:.2%})")
    
    # DTF chi2 cut - only apply if the field exists
    try:
        data = data[data['Bu_DTF_chi2'] < 30]
        print(f"After DTF chi2 cut: {len(data)}/{initial_count} ({len(data)/initial_count:.2%})")
    except:
        print("Skipping DTF chi2 cut (field not found)")
    
    # Impact Parameter Chi2 - only apply if the field exists
    try:
        data = data[data['Bu_IPCHI2_OWNPV'] < 10]
        print(f"After IP chi2 cut: {len(data)}/{initial_count} ({len(data)/initial_count:.2%})")
    except:
        print("Skipping IP chi2 cut (field not found)")
    
    # Flight Distance Chi2 - only apply if the field exists
    try:
        data = data[data['Bu_FDCHI2_OWNPV'] > 175]
        print(f"After B FD chi2 cut: {len(data)}/{initial_count} ({len(data)/initial_count:.2%})")
    except:
        print("Skipping B FD chi2 cut (field not found)")
    
    print(f"Final selected events: {len(data)}/{initial_count} ({len(data)/initial_count:.2%})")
    return data

In [7]:
# mc_selection_ll = select(mc_ll, track_type='LL')  # or 'DD' for Downstream-Downstream
# mc_selection_dd = select(mc_dd, track_type='DD')  # or 'LL' for Long-Long

mc_selection_ll = mc_ll
mc_selection_dd = mc_dd

In [8]:
from utils.mc import *
import json
import os
import contextlib

# Context manager to patch title in plots
@contextlib.contextmanager
def with_title_patch(new_title):
    """Context manager to temporarily patch the title in RooFit frames"""
    original_frame = ROOT.RooPlot.frame
    
    def patched_frame(self, *args, **kwargs):
        if 'Title' in kwargs:
            kwargs['Title'] = new_title
        else:
            # Find RooFit.Title in args
            for i, arg in enumerate(args):
                if isinstance(arg, ROOT.RooFit.RooCmdArg) and arg.GetName() == "Title":
                    args = list(args)
                    args[i] = ROOT.RooFit.Title(new_title)
                    args = tuple(args)
                    break
        return original_frame(self, *args, **kwargs)
    
    try:
        ROOT.RooPlot.frame = patched_frame
        yield
    finally:
        ROOT.RooPlot.frame = original_frame

def analyze_track_type(mc_selection, track_type, models=None):
    """
    Run analysis on a specific track type
    
    Parameters:
    -----------
    mc_selection : array
        MC data selection for this track type
    track_type : str
        Track type ('DD' or 'LL')
    models : list or None
        List of models to run, defaults to ['gaussian', 'double_gaussian']
    
    Returns:
    --------
    dict of results
    """
    if models is None:
        models = ['gaussian', 'double_gaussian']
    
    output_dir = f"output_{track_type}/"
    
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created output directory: {output_dir}")
    
    results = {}
    
    # Run the specified models
    if 'gaussian' in models:
        print(f"\n=== Fitting with Gaussian + Poly2 model ({track_type}) ===")
        with with_title_patch(f"B+ Mass Shape - Gaussian+Poly2 Model - {track_type}"):
            results['gaussian_poly2'] = extract_signal_shape_with_background(
                mc_selection,
                true_id_branch='Bu_TRUEID',
                mass_branch='Bu_MM',
                output_dir=output_dir
            )
    
    if 'double_gaussian' in models:
        print(f"\n=== Fitting with Double Gaussian + Poly2 model ({track_type}) ===")
        with with_title_patch(f"B+ Mass Shape - Double Gaussian + Poly2 Model - {track_type}"):
            results['double_gaussian_poly2'] = extract_signal_shape_double_gaussian_with_background(
                mc_selection,
                true_id_branch='Bu_TRUEID',
                mass_branch='Bu_MM',
                output_dir=output_dir
            )
    
    if 'crystal_ball' in models:
        print(f"\n=== Fitting with Crystal Ball + Poly2 model ({track_type}) ===")
        with with_title_patch(f"B+ Mass Shape - Crystal Ball + Poly2 Model - {track_type}"):
            results['crystal_ball_poly2'] = extract_signal_shape_cb_with_background(
                mc_selection,
                true_id_branch='Bu_TRUEID',
                mass_branch='Bu_MM',
                output_dir=output_dir
            )
    
    if 'double_crystal_ball' in models:
        print(f"\n=== Fitting with Double Crystal Ball + Poly2 model ({track_type}) ===")
        with with_title_patch(f"B+ Mass Shape - Double Crystal Ball + Poly2 Model - {track_type}"):
            results['double_crystal_ball_poly2'] = extract_signal_shape_double_cb_with_background(
                mc_selection,
                true_id_branch='Bu_TRUEID',
                mass_branch='Bu_MM',
                output_dir=output_dir
            )
    
    # Find best model based on χ²/ndf
    best_model = min(results.items(), key=lambda x: x[1]['chi2_ndf'])
    print(f"\nBest model for {track_type}: {best_model[0]} with χ²/ndf = {best_model[1]['chi2_ndf']:.3f}")
    
    # Save all results to a JSON file for reference
    with open(f"{output_dir}/model_comparison.json", 'w') as f:
        comparison = {
            model_name: {"chi2_ndf": params["chi2_ndf"]}
            for model_name, params in results.items()
        }
        json.dump(comparison, f, indent=4)
    
    return results

# Define which models we want to run
models_to_run = ['gaussian', 'double_gaussian']
# Optional: add more models if needed
# models_to_run = ['gaussian', 'double_gaussian', 'crystal_ball', 'double_crystal_ball']

# Analyze LL track type
print("\n========== ANALYZING LONG-LONG TRACKS ==========")
ll_results = analyze_track_type(mc_selection_ll, 'LL', models_to_run)

# Analyze DD track type
print("\n========== ANALYZING DOWNSTREAM-DOWNSTREAM TRACKS ==========")
dd_results = analyze_track_type(mc_selection_dd, 'DD', models_to_run)

# Compare results between track types
print("\n========== TRACK TYPE COMPARISON ==========")
for model in set(ll_results.keys()).intersection(dd_results.keys()):
    print(f"\nModel: {model}")
    print(f"LL χ²/ndf: {ll_results[model]['chi2_ndf']:.3f}")
    print(f"DD χ²/ndf: {dd_results[model]['chi2_ndf']:.3f}")
    
    # Compare resolutions
    if model in ['gaussian_poly2', 'crystal_ball_poly2']:
        ll_res = ll_results[model]['sigma']['value']
        ll_res_err = ll_results[model]['sigma']['error']
        dd_res = dd_results[model]['sigma']['value']
        dd_res_err = dd_results[model]['sigma']['error']
        print(f"LL resolution: {ll_res:.2f} ± {ll_res_err:.2f} MeV/c²")
        print(f"DD resolution: {dd_res:.2f} ± {dd_res_err:.2f} MeV/c²")
    elif model in ['double_gaussian_poly2', 'double_crystal_ball_poly2']:
        ll_res = ll_results[model]['effective_sigma']['value'] if 'effective_sigma' in ll_results[model] else ll_results[model]['sigma']['value']
        ll_res_err = ll_results[model]['effective_sigma']['error'] if 'effective_sigma' in ll_results[model] else ll_results[model]['sigma']['error']
        dd_res = dd_results[model]['effective_sigma']['value'] if 'effective_sigma' in dd_results[model] else dd_results[model]['sigma']['value']
        dd_res_err = dd_results[model]['effective_sigma']['error'] if 'effective_sigma' in dd_results[model] else dd_results[model]['sigma']['error']
        print(f"LL resolution: {ll_res:.2f} ± {ll_res_err:.2f} MeV/c²")
        print(f"DD resolution: {dd_res:.2f} ± {dd_res_err:.2f} MeV/c²")

Welcome to JupyROOT 6.28/00

========== ANALYZING LONG-LONG TRACKS ==========
Created output directory: output_LL/

=== Fitting with Gaussian + Poly2 model (LL) ===
Extracting shape using Gaussian+Poly2 model
Number of events: 30616
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 c0           1.00000e-01  2.00000e-01   -1.00000e+00  1.00000e+00
     2 c1           1.00000e-01  2.00000e-01   -1.00000e+00  1.00000e+00
     3 c2           1.00000e-01  2.00000e-01   -1.00000e+00  1.00000e+00
     4 mean         5.27900e+03  6.00000e+00    5.24900e+03  5.30900e+03
     5 nbkg         1.51595e+04  6.06380e+03    0.00000e+00  6.06380e+04
     6 nsig         1.51595e+04  6.06380e+03    0.00000e+00  6.06380e+04
     7 sigma        1.50000e+01  2.50000e+00    5.00000e+00  3.00000e+01
 **********
 **    3 **SET ERR         0.5
 **********
 **********
 **   

Info in <TCanvas::Print>: pdf file output_LL//b_mass_shape_MC_Gauss_Pol2.pdf has been created
Info in <TCanvas::Print>: pdf file output_LL//b_mass_shape_MC_DoubleGauss_Pol2.pdf has been created
Info in <TCanvas::Print>: pdf file output_DD//b_mass_shape_MC_Gauss_Pol2.pdf has been created
Info in <TCanvas::Print>: pdf file output_DD//b_mass_shape_MC_DoubleGauss_Pol2.pdf has been created


level p.d.f evaluates to NaN @ !refCoefNorm=(mass = 5201.25), !pdfs=(gauss = 4.2605e-30/17.1415,poly = -1.09647/258.712), !coefficients=(nsig = 58027.2,nbkg = 1882.93)
     getLogVal() top-level p.d.f evaluates to NaN @ !refCoefNorm=(mass = 5203.75), !pdfs=(gauss = 2.79846e-28/17.1415,poly = -0.800537/258.712), !coefficients=(nsig = 58027.2,nbkg = 1882.93)
     getLogVal() top-level p.d.f evaluates to NaN @ !refCoefNorm=(mass = 5206.25), !pdfs=(gauss = 1.60818e-26/17.1415,poly = -0.520513/258.712), !coefficients=(nsig = 58027.2,nbkg = 1882.93)
     getLogVal() top-level p.d.f evaluates to NaN @ !refCoefNorm=(mass = 5208.75), !pdfs=(gauss = 8.08555e-25/17.1415,poly = -0.25604/258.712), !coefficients=(nsig = 58027.2,nbkg = 1882.93)
     getLogVal() top-level p.d.f evaluates to NaN @ !refCoefNorm=(mass = 5211.25), !pdfs=(gauss = 3.55664e-23/17.1415,poly = -0.00676274/258.712), !coefficients=(nsig = 58027.2,nbkg = 1882.93)
RooChebychev::poly[ x=mass coefList=(c0,c1,c2) ]
     p.d.f value i